In [13]:
%%capture
!pip3 install google-auth google-cloud-dialogflow-cx

In [14]:
# data manipulation modules
import pandas as pd
import numpy as np

# general python modules
import json
import csv
import io 
import os

# dialogflow cx imports
from dfcx_scrapi.core.agents import Agents
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.core.flows import Flows
from dfcx_scrapi.core.pages import Pages
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions
from dfcx_scrapi.tools.maker_util import MakerUtil

# connection to g-sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

from google.cloud import dialogflowcx_v3beta1


In [15]:
# GCP Service Account Scopes 
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']


#authorize sa
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/christophertrauco/.config/secrets/cx_admin_sa.json', scope)
client = gspread.authorize(credentials)
spreadsheet_key = '18qvh0uWkA0z7S5CC3xTdvD2wkhsmnINjgxUWmuoCo7M'
sh = client.open_by_key(spreadsheet_key)


In [26]:
%%capture
%%sh
./etl.sh


In [27]:
# gcp project_id mappings to g-sheets

file = 'projects'

data = f'{file}.csv'
df = pd.read_csv(data)

# write gcp projects to sad dfcx workbook

wks_name = sh.worksheet('Mappings').acell('B1').value
cell_of_start_df = 'A1'
# upload the dataframe of the clients we want to delete
d2g.upload(df,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=False)
print ('The sheet is updated successfully')

The sheet is updated successfully


In [28]:
project_id = sh.worksheet('Main').acell('B1').value
display_name = sh.worksheet('Main').acell('B2').value
gcp_region = sh.worksheet('Main').acell('B3').value
zone = sh.worksheet('Main').acell('B4').value
language = sh.worksheet('Main').acell('B5').value
time_zone = sh.worksheet('Main').acell('B6').value
creds_path = sh.worksheet('Main').acell('B8').value

In [31]:
# gcp list all agents within scope of mapped project_id
def list_agents():
    # Create a client
    client = dialogflowcx_v3beta1.AgentsClient()

    # Initialize request argument(s)
    request = dialogflowcx_v3beta1.ListAgentsRequest(
        parent="projects/bot1-404120/locations/global",
    )

    # Make the request
    page_result = client.list_agents(request=request)

    # Handle the response
    for response in page_result:
        print(response)

        # # [END dialogflow_v3beta1_generated_Agents_ListAgents_sync]

In [33]:
df = pd.read_csv('agents.csv')
# print(df)

wks_name = sh.worksheet('Mappings').acell('B2').value
cell_of_start_df = 'A1'
# upload the dataframe of the clients we want to delete
d2g.upload(df,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

The sheet is updated successfully
